<a href="https://colab.research.google.com/github/Saulrega/pytorch/blob/main/Cycle_Gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/Deep learning pytorch/'
!ls

/content/drive/MyDrive/Colab Notebooks/Deep learning pytorch
'Capas de red neuronal.ipynb'  'Documento sin título.gdoc'    Tensores.ipynb
'Cycle Gan.ipynb'	       'Primera red neuronal.ipynb'
 dl-pytorch		       'Setup inicial.ipynb'


In [ ]:
!git clone https://github.com/aitorzip/PyTorch-CycleGAN.git

Cloning into 'PyTorch-CycleGAN'...
remote: Enumerating objects: 60, done.
remote: Total 60 (delta 0), reused 0 (delta 0), pack-reused 60
Unpacking objects: 100% (60/60), done.


In [ ]:
%cd 'PyTorch-CycleGAN'

/content/drive/MyDrive/Colab Notebooks/Deep learning pytorch/PyTorch-CycleGAN


In [ ]:
%%sh
sh ./download_dataset summer2winter_yosemite

In [ ]:
!mv datasets/summer2winter_yosemite /content/drive/MyDrive/Colab\ Notebooks/Deep\ learning\ pytorch/dl-pytorch/datasets/

In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/Deep learning pytorch/dl-pytorch/datasets'

64x64_SIGNS	 cifar-10-batches-py	 summer2winter_yosemite
64x64_SIGNS.zip  cifar-10-python.tar.gz


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
  def __init__(self, in_features):
    super(ResidualBlock, self).__init__()
    
    conv_block = [ nn.ReflectionPad2d(1), #mejor padding
                   nn.Conv2d(in_features, in_features, 3),
                   nn.InstanceNorm2d(in_features), #BN para GANS
                   nn.ReLU(True),
                   nn.ReflectionPad2d(1), #mejor para conservar distribucion
                   nn.Conv2d(in_features, in_features, 3),
                   nn.InstanceNorm2d(in_features)
                 ]
    
    self.conv_block = nn.Sequential(*conv_block)
  def forward(self, x):
    return self.conv_block(x) + x #una idea poderosa

In [ ]:
class Generator(nn.Module):
  def __init__(self, input_nc, output_nc, n_residual_blocks=9):
    super(Generator,self).__init__()
    
    # Bloqueconvolucional
    model = [ nn.ReflectionPad2d(3),
              nn.Conv2d(input_nc, 64, 7), # I - 7 + 6 /1 +1 = I
              nn.InstanceNorm2d(64),
              nn.ReLU(True)
            ]
    
    in_features = 64
    out_features = in_features * 2
    
    #Encoding
    for _ in range(2):
      model += [ nn.Conv2d(in_features, out_features, 3, stride=2, padding=1), #I/2
                 nn.InstanceNorm2d(out_features),
                 nn.ReLU(True)
               ]
      in_features = out_features
      out_features = in_features*2

    #transformaciones residuales
    
    for _ in range(n_residual_blocks):
      model += [ResidualBlock(in_features)]
    
    #decoding
    
    out_features = in_features//2
    for _ in range(2):
      model += [ nn.ConvTransose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1), #2I
                 nn.InstanceNorm2d(out_features),
                 nn.ReLU(True)
               ]
      in_features = out_features
      out_feature = in_features //2
      
    
    #salida
    model += [ nn.ReflectionPad2d(3),
               nn.Conv2d(64, output_nc, 7), #I
               nn.Tanh()
            ]
      
    self.model = nn.Sequential(*model)
      
    def forward(self,x):
      return self.model(x)

In [ ]:
class Discriminator(nn.Module):
  "PatchGAN: discrimina estilo o textura"
  def __init__(self, input_nc):
    super(Discriminator, self).__init__()
    
    model = [ nn.Conv2d(input_nc, 64, 4, stride=2, padding=1), #I/2
              nn.LeakyReLU(0.2, in_place=True)
            ]
    
    model += [ nn.Conv2d(64, 128, 4, stride=2, padding=1), #I/2
               nn.InstanceNorm2d(128),
               nn.LeakyReLU(0.2, in_place=True)
             ]
    
    model += [ nn.Conv2d(128, 256, 4, stride=2, padding=1), #I/2
               nn.InstanceNorm2d(256),
               nn.LeakyReLU(0.2, in_place=True)
             ]
    
    model += [ nn.Conv2d(256, 512, 4, padding=1), #I-1
               nn.InstanceNorm2d(512),
               nn.LeakyReLU(0.2, in_place=True)
             ]
    
    # Flatten
    model += [nn.Conv2d(512, 1, 4, padding=1)] #I-1
    
    self.model = nn.Sequential(*model)
    
  def forward(self, x):
    x = self.model(x)
    return F.avg_pool2d(x, x.size()[2:]).view(x.size()[0], -1)

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Deep learning pytorch/dl-pytorch/')

In [ ]:
!ls /content/drive/MyDrive/Colab\ Notebooks/Deep\ learning\ pytorch/dl-pytorch/

datasets  optimizers_viz.py  plot_helpers.py  __pycache__  utils.py


In [ ]:
!pip install livelossplot

In [ ]:
!pip install visdom

In [ ]:
import glob 
import random 
import os 
import itertools
from PIL import Image

import torch

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

from utils import ReplayBuffer, Logger

from livelossplot import PlotLosses

In [ ]:
class ImageDataset(Dataset):
  def __init__(self, base_dir, transform=None, split='train'):
    self.transform = transforms.Compose(transform)
    self.files_A = sorted(glob.glob(os.path.join(base_dir, '{}/A/*.*'.format(split))))
    self.files_B = sorted(glob.glob(os.path.join(base_dir, '{}/B/*.*'.format(split))))
    
  def __len__(self):
    return max(len(self.files_A), len(self.files_B))
  
  def __getitem__(self,idx):
    image_A = self.transform(Image.open(self.files_A[idx]))
    image_B = self.transform(Image.open(self.files_B[random.randint(0,len(self.files_B)-1)]))
    return {'A': image_A, 'B': image_B}

In [ ]:
epoch=0
n_epochs = 200
batch_size = 2 #4
lr = 0.0002
size = 256
input_nc = 3
output_nc = 3
decay_epoch= 100 #pending

cuda = True
n_cpu = 8

base_dir = '/content/drive/MyDrive/Colab Notebooks/Deep learning pytorch/dl-pytorch/datasets/summer2winter_yosemite/'

In [ ]:
device = torch.device('cuda' if cuda else 'cpu')

def weights_init_normal(m):
  if isinstance(m, nn.Conv2d):
    torch.nn.init.normal(m.weight.data, 0.0, 0.02)
  elif isinstance(m, nn.BatchNorm2d):
    torch.nn.init.normal(m.weight.data, 1.0, 0.02)
    torch.nn.init.constant(m.bias, 0.0)
    
netG_A2B = Generator(input_nc, output_nc)
netG_B2A = Generator(input_nc, output_nc)
netD_A = Discriminator(input_nc)
netD_B = Discriminator(input_nc)

netG_A2B.apply(weights_init_normal)
netG_B2A.apply(weights_init_normal)
netD_A.apply(weights_init_normal)
netD_B.apply(weights_init_normal)

if cuda:
  netG_A2B.to(device)
  netG_B2A.to(device)
  netD_A.to(device)
  netD_B.to(device)
  
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

## Instanciando optimizadores y schedulers

In [ ]:
optimizer_G = torch.optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()),
                              lr=lr, betas=(0.5, 0.999))
optimizer_D_A = torch.optim.Adam(netD_A.parameters(), lr=lr, betas=(0.5,0.999))
optimizer_D_B = torch.optim.Adam(netD_B.parameters(), lr=lr, betas=(0.5,0.999))

#schedulers (actualizar el learning rate de forma dinamica durante el entrenamiento)

class LambdaLR():
  def __init__(self, n_epochs, offset, decay_start_epoch):
    assert ((n_epochs - decay_start_epoch) > 0)
    self.n_epochs = n_epochs
    self.offset = offset
    self.decay_start_epoch = decay_start_epoch
    
  def step(self, epoch):
    return 1 - max(0, epoch + self.offset - self.decay_start_epoch)/(self.n_epochs - self.decay_start_epoch)

lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, 
                                                   lr_lambda=LambdaLR(n_epochs,epoch,decay_epoch).step)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, 
                                                     lr_lambda=LambdaLR(n_epochs,epoch,decay_epoch).step)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, 
                                                     lr_lambda=LambdaLR(n_epochs,epoch,decay_epoch).step)


## Calculando las perdidas

In [ ]:
#inputs y targets

Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor
target_real = Tensor(batch_size).fill_(1.0)
target_fake = Tensor(batch_size).fill_(0.0)

fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

#Dataloader

transform = [ transforms.Resize(int(size*1.12), Image.BICUBIC),
              transforms.RandomCrop(size),
              transforms.RandomHorizontalFlip(),
              transforms.ToTensor(),
              transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
            ]

dataloader = DataLoader(ImageDataset(base_dir,transform=transform),
                       batch_size=batch_size, shuffle=True, num_workers=n_cpu, drop_last=True)

def Gen_GAN_loss(G, D, real, loss, target_real):
  fake = G(real)
  pred_fake = D(fake)
  L = loss(pred_fake, target_real)
  return L, fake

def cycle_loss(G1, G2, real, loss):
  recovered = G2(G1(real))
  L = loss(recovered, real)
  return L

def identity_loss(G, real, loss):
  same = G(real)
  L = loss(same,real)
  return L

def Disc_GAN_loss(D2, fake2, real2, fake_2_buffer, loss, target_real, target_fake):
  pred_real = D2(real2)
  loss_D2_real = loss(pred_real, target_real)
  
  fake2 = fake_2_buffer.push_and_pop(fake2)
  pred_fake = D2(fake2.detach())
  loss_D2_fake = loss(pred_fake, target_fake)
  loss_D2 = (loss_D2_real + loss_D2_fake) * 0.5
  return loss_D2

## Entrenamiento y visualización del entrenamiento

In [ ]:
!pip install livelossplot

In [ ]:
from livelossplot import PlotLosses
from utils import Logger

logger = Logger(n_epochs, len(dataloader), epoch=epoch)
liveloss= PlotLosses()

In [ ]:
for epoch in range(epoch, n_epochs):
  for i, batch in enumerate(dataloader):
    real_A = batch['A'].to(device)
    real_B = batch['B'].to(device)
    
    # Generativas
    optimizer_G.zero_grad()
    
    loss_GAN_A2B, fake_B = Gen_GAN_loss(netG_A2B, netD_B, real_A, criterion_GAN, target_real)
    loss_GAN_B2A, fake_A = Gen_GAN_loss(netG_B2A, netD_A, real_B, criterion_GAN, target_real)
    
    loss_cycle_ABA = cycle_loss(netG_A2B, netG_B2A, real_A, criterion_cycle)
    loss_cycle_BAB = cycle_loss(netG_B2A, netG_A2B, real_B, criterion_cycle)
    
    loss_identity_A = identity_loss(netG_B2A, real_A, criterion_identity)
    loss_identity_B = identity_loss(netG_A2B, real_B, criterion_identity)
    
    loss_G = (loss_GAN_A2B + loss_GAN_B2A) + 10.0*(loss_cycle_ABA + loss_cycle_BAB) + 5.0 *(loss_identity_A + loss_identity_B)
    loss_G.backward()
    
    optimizer_G.step()
    
    #Discriminativas
    optimizer_D_A.zero_grad()
    
    loss_D_A = Disc_GAN_loss(netD_A, fake_A, real_A, fake_A_buffer, criterion_GAN, target_real, target_fake)
    loss_D_A.backward()
    optimizer_D_A.step()
    
    optimizer_D_B.zero_grad()
    
    loss_D_B = Disc_GAN_loss(netD_B, fake_B, real_B, fake_B_buffer, criterion_GAN, target_real, target_fake)
    loss_D_B.backward()
    optimizer_D_B.step()
    
    log_values = {'loss_G': loss_G,
                  'loss_G_identity': (loss_identity_A + loss_identity_B),
                  'loss_G_GAN': (loss_GAN_A2B + loss_GAN_B2A),
                  'loss_G_cyle': loss_cycle_ABA + loss_cycle_BAB,
                  'loss_D': loss_D_A + loss_D_B
                 }
    logger.log(log_values, images={'real_A': real_A,'real_B': real_B, 'fake_A': fake_A,'fake_B': fake_B  })
  
  liveloss.update(log_values)
  liveloss.draw()
    
  lr_scheduler_G.step()
  lr_scheduler_D_A.step()
  lr_scheduler_D_B.step()
  